In [ ]:
import sys
print(sys.version)

3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]


In my Spark pipeline, Spark is leveraged for distributed processing of the social network data. It uses RDD transformations like flatMap and reduceByKey to efficiently identify all pairs of mutual friends. Subsequently, Spark performs joins and filters to exclude existing friendships, retaining only potential recommendations. Finally, it groups these recommendations by user and sorts them to present the top suggestions.


In [ ]:
# !pip install pyspark==3.3.2
# !pip install pyspark
!pip install -U -q PyDrive
!pip uninstall -y PyDrive
!pip install -U PyDrive2
!apt install openjdk-8-jdk-headless -qq
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

  Preparing metadata (setup.py) ... done
Found existing installation: PyDrive 1.3.1
Uninstalling PyDrive-1.3.1:
  Successfully uninstalled PyDrive-1.3.1
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 39.7 MB/39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u452-ga%7eus1-0ubuntu1%7e22.04_amd64.deb  404  Not Found [IP: 91.189.92.23 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u452-ga%

In [ ]:
# !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

openjdk version "11.0.27" 2025-04-15
OpenJDK Runtime Environment (build 11.0.27+6-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.27+6-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1heo2wadgXyS54zh-XNukAyq8saDzWS6H'

# Download
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

In [ ]:
from pyspark import SparkContext
from itertools import combinations

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
        .appName("ColabSpark")
        .master("local[*]")
        .config("spark.ui.showConsoleProgress", "true")
        .config("spark.driver.memory", "4g")
        .getOrCreate()
)

spark


In [ ]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setAppName("PeopleYouMightKnow")

sc = SparkContext.getOrCreate()

lines = sc.textFile("soc-LiveJournal1Adj.txt")

In [ ]:
# sc.parallelize([1, 2, 3]).map(lambda x: x * 10).collect()

lines.take(5)


['0\t1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94',
 '1\t0,5,20,135,2409,8715,8932,10623,12347,12846,13840,13845,14005,20075,21556,22939,23520,28193,29724,29791,29826,30691,31232,31435,32317,32489,34394,35589,35605,35606,35613,35633,35648,35678,38737,43447,44846,44887,49226,49985,623,629,4999,6156,13912,14248,15190,17636,19217,20074,27536,29481,29726,29767,30257,33060,34250,34280,34392,34406,34418,34420,34439,34450,34651,45054,49592',
 '2\t0,117,135,1220,2755,12453,24539,24714,41456,45046,49927,6893,13795,16659,32828,41878',
 '3\t0,12,41,55,1532,12636,13185,27552,38737',
 '4\t0,8,14,15,18,27,72,80,15326,19068,19079,24596,42697,46126,74,77,33269,38792,38822']

In [ ]:
users_friends = lines.map(
    lambda line: (
        int(line.split('\t')[0]),
        [int(f) for f in line.split('\t')[1].split(',') if f.strip() != ""]
        if len(line.split('\t')) > 1 else []
    )
)


In [ ]:
print("USERS_FRIENDS:", users_friends.take(5))


USERS_FRIENDS: [(0, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]), (1, [0, 5, 20, 135, 2409, 8715, 8932, 10623, 12347, 12846, 13840, 13845, 14005, 20075, 21556, 22939, 23520, 28193, 29724, 29791, 29826, 30691, 31232, 31435, 32317, 32489, 34394, 35589, 35605, 35606, 35613, 35633, 35648, 35678, 38737, 43447, 44846, 44887, 49226, 49985, 623, 629, 4999, 6156, 13912, 14248, 15190, 17636, 19217, 20074, 27536, 29481, 29726, 29767, 30257, 33060, 34250, 34280, 34392, 34406, 34418, 34420, 34439, 34450, 34651, 45054, 49592]), (2, [0, 117, 135, 1220, 2755, 12453, 24539, 24714, 41456, 45046, 49927, 6893, 13795, 16659, 32828, 41878]), (3, [0, 12, 41, 55, 1532, 12636, 13185, 27552, 3

In [ ]:
users_friends.take(5)


[(0,
  [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94]),
 (1,
  [0,
   5,
   20,
   135,
   2409,
   8715,
   8932,
   10623,
   12347,
   12846,
   13840,
   13845,
   14005,
   20075,
   21556,
   22939,
   23520,
   28193,
   29724,
   29791,
   29826,
   30691,
   31232,
   31435,
   32317,
   32489,
   34394,
   35589,
   35605,
   35606,
   35613,
   35633,
   35648,
   35678,
   38737,
   43

In [ ]:
for u, friends in users_friends.take(5):
    print(f"User {u} has friends: {friends}")


User 0 has friends: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]
User 1 has friends: [0, 5, 20, 135, 2409, 8715, 8932, 10623, 12347, 12846, 13840, 13845, 14005, 20075, 21556, 22939, 23520, 28193, 29724, 29791, 29826, 30691, 31232, 31435, 32317, 32489, 34394, 35589, 35605, 35606, 35613, 35633, 35648, 35678, 38737, 43447, 44846, 44887, 49226, 49985, 623, 629, 4999, 6156, 13912, 14248, 15190, 17636, 19217, 20074, 27536, 29481, 29726, 29767, 30257, 33060, 34250, 34280, 34392, 34406, 34418, 34420, 34439, 34450, 34651, 45054, 49592]
User 2 has friends: [0, 117, 135, 1220, 2755, 12453, 24539, 24714, 41456, 45046, 49927, 6893, 13795, 16659, 32828, 41878]
User 3 has friends: [0

In [ ]:
def mutual_pairs(user, friends):
    out = []
    for f1, f2 in combinations(friends, 2):
        out.append(((f1, f2), 1))
        out.append(((f2, f1), 1))
    return out


In [ ]:
pairs = users_friends.flatMap(lambda x: mutual_pairs(x[0], x[1]))

In [ ]:
pairs.take(50)

[((1, 2), 1),
 ((2, 1), 1),
 ((1, 3), 1),
 ((3, 1), 1),
 ((1, 4), 1),
 ((4, 1), 1),
 ((1, 5), 1),
 ((5, 1), 1),
 ((1, 6), 1),
 ((6, 1), 1),
 ((1, 7), 1),
 ((7, 1), 1),
 ((1, 8), 1),
 ((8, 1), 1),
 ((1, 9), 1),
 ((9, 1), 1),
 ((1, 10), 1),
 ((10, 1), 1),
 ((1, 11), 1),
 ((11, 1), 1),
 ((1, 12), 1),
 ((12, 1), 1),
 ((1, 13), 1),
 ((13, 1), 1),
 ((1, 14), 1),
 ((14, 1), 1),
 ((1, 15), 1),
 ((15, 1), 1),
 ((1, 16), 1),
 ((16, 1), 1),
 ((1, 17), 1),
 ((17, 1), 1),
 ((1, 18), 1),
 ((18, 1), 1),
 ((1, 19), 1),
 ((19, 1), 1),
 ((1, 20), 1),
 ((20, 1), 1),
 ((1, 21), 1),
 ((21, 1), 1),
 ((1, 22), 1),
 ((22, 1), 1),
 ((1, 23), 1),
 ((23, 1), 1),
 ((1, 24), 1),
 ((24, 1), 1),
 ((1, 25), 1),
 ((25, 1), 1),
 ((1, 26), 1),
 ((26, 1), 1)]

In [ ]:
pairs = pairs.filter(lambda x: x[0][0] != x[0][1])

In [ ]:
print("pairs (filtered):", pairs.take(10))

pairs (filtered): [((1, 2), 1), ((2, 1), 1), ((1, 3), 1), ((3, 1), 1), ((1, 4), 1), ((4, 1), 1), ((1, 5), 1), ((5, 1), 1), ((1, 6), 1), ((6, 1), 1)]


In [ ]:
mutual_counts = pairs.reduceByKey(lambda a, b: a + b)

In [ ]:
# print("mutual_counts:", mutual_counts.take(2))

In [ ]:
edges = users_friends.flatMap(
    lambda x: [((x[0], f), 1) for f in x[1]] +
              [((f, x[0]), 1) for f in x[1]]
)

In [ ]:
# print("edges:", edges.take(3))

In [ ]:
candidates = mutual_counts.leftOuterJoin(edges)

In [ ]:
filtered = candidates.filter(lambda x: x[1][1] is None).map(lambda x: (x[0], x[1][0]))

In [ ]:
# print("filtered:", filtered.take(3))

In [ ]:
user_recs = filtered.map(
    lambda x: (x[0][0], (x[0][1], x[1]))
).groupByKey()

In [ ]:
# print("user_recs:", user_recs.take(3))

In [ ]:

def top_10(recs):
    return sorted(list(recs), key=lambda x: (-x[1], x[0]))[:10]

final_recs = user_recs.mapValues(top_10)


In [ ]:
formatted = final_recs.map(
    lambda x: f"{x[0]}\t{','.join(str(rec[0]) for rec in x[1])}"
)


In [ ]:
# formatted.take(5)

In [ ]:
formatted.filter(lambda x: x.startswith("11\t")).take(1)


['11\t27552,7785,27573,27574,27589,27590,27600,27617,27620,27667']

In [ ]:
user_ids = [924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]


filtered_recommendations = formatted.filter(lambda x: int(x.split('\t')[0]) in user_ids)

# Collected and printed the recommendations
for rec in filtered_recommendations.collect():
    print(rec)

9020	9021,9016,9017,9022,317,9023
924	439,2409,6995,11860,15416,43748,45881
9992	9987,9989,35667,9991
9021	9020,9016,9017,9022,317,9023
9993	9991,13134,13478,13877,34299,34485,34642,37941
8941	8943,8944,8940
8942	8939,8940,8943,8944
9022	9019,9020,9021,317,9016,9017,9023
9990	13134,13478,13877,34299,34485,34642,37941
9019	9022,317,9023


In [ ]:
formatted.saveAsTextFile("/content/output")